In [ ]:
from pycalphad import Database, equilibrium, variables as v, calculate
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)

In [ ]:
# constants:
P = 101325

# Компоненты и фазы

In [ ]:
import random

random.seed(10)
all_components = list(db.elements)
all_phases =  list(db.phases)
all_phases.sort()
all_components.sort()
print('Количество фаз =' + str(len(all_phases)))
print('Количество компонент =' + str(len(all_components)))
print(all_components)
# components = random.sample(all_components, 8)
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']

print(components)


# Бинарные фазы для компонент

In [ ]:
counter = 0 
components_phases = {}
for comp in components:
    components_phases[comp] = set()
for item in db.phases.items():
    # item[0] - name, item[1] - content
    # item[1] - name, constituents, sublattices, model_hints
    if len(item[1].constituents)>4:
        counter += 1
    
    for species in item[1].constituents[0]:
        # name, constituents, charge
        for phase in components_phases.keys():
            if phase in species.name:
                components_phases[phase].add(item[0])

print(counter)
print(components_phases)

for comp, phases in components_phases.items():
    mask = [1 if phase in phases else 0 for phase in all_phases]
    components_phases[comp] = int(''.join(map(str, mask)), 2)

In [ ]:
for comp, phases in components_phases.items():
    print(f'{comp}: {bin(phases)}')

In [ ]:
def getPhases(int_mask):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]
def getAllPhases(comp_phases):
    int_mask = 0
    for phases in comp_phases.values():
        int_mask |= phases
    return getPhases(int_mask)
# print(getAllPhases(components_phases))
print(getPhases(components_phases['SE']))

In [ ]:
for comp, phases in components_phases.items():
    print(comp)
    print(getPhases(phases))
    for phase in getPhases(phases):
        if str(phase) in db.symbols:
            print(f'{phase} in {db.symbols}')

# Функции

In [ ]:
import re

# read the text file
text = ""
function_start = False
with open(db_name, 'r') as f:
    for line in f:
        if "Standard functions for the elements" in line:
            function_start = True
        if "Data for the elements from unary database" in line:
            function_start = False

        if function_start:
            text += line

component_functions = {}
# find all the component names in the text
component_names = re.findall(r'\$ ([A-Z][a-z]*)', text)
for component in component_names:
    # find the text between the component name and the next component name
    pattern = r'\$ {}.*?\$ [A-Z][a-z]*'.format(component)
    component_text = re.search(pattern, text, re.DOTALL).group()
    
    # find all the function names in the component text
    function_names = re.findall(r'FUNCTION ([A-Z]*)', component_text)
    
    component_functions[component] = function_names
component_functions = {k.upper():v for k,v in component_functions.items()}

In [ ]:
# print the functions for each component
for component, functions in component_functions.items():
    if len(functions) > 0:
        print(f"{component} : {functions}")

In [ ]:
functions = set()
for component in components:
    if str(component) in component_functions.keys():
        functions.update(component_functions[component])
    else:
        print(f'Check - {component}')
print(functions)

In [ ]:
from symengine import Piecewise, Symbol, lib
import math

temp = 1200
T = Symbol('T')
print(db.symbols['GBCCGD'])
print(type(temp))
expr = db.symbols['GBCCGD'].subs(T, temp)
print(expr)
# print(db.symbols['GBCCGD'].subs(T, 800).simplify())
# print (math.log(14))


In [ ]:
function_value = {}
func = 'GBCCGD'
for func in functions:
    expression = Piecewise()
    print(func)
    if func not in db.symbols:
        print('Wrong function: ' + str(func))
        function_value[func] = None
        continue
    
    for arg in db.symbols[func].args:
        # condition = arg[0].subs(T, temp)
        # expression = arg[1].subs(T, temp)
        # result.append([condition, expression])
        if arg.is_Add:
            expression = arg.subs(T, temp).evalf()
        elif arg.is_Float or arg.is_Integer:
            expression = arg
        else:
            bool_res = False
            if len(arg.atoms()) > 0:
                bool_res = arg.subs(T, temp)
            if arg.is_Boolean:
                bool_res = arg
            if bool_res: 
                function_value[func] = expression
                break

print(function_value)

# Расчет фазовых равновесий

In [83]:
necessary_component = 'VA'
temp = 1200
if necessary_component not in components:
    components.append(necessary_component)

conditions = {v.T:temp,v.P:P}
# TODO: make random amount of component
amount = 1.0/len(components)
for comp in components:
    if comp != necessary_component:
        conditions[v.X(comp)] = amount
eq = equilibrium(db,components,getAllPhases(components_phases),conditions, verbose=True)
gibbs_energy = eq.GM

Components: AL AL1 AL1AU AL1AU2 AL1AU4 AL2 AL2AU AL2AU5 AU HG RH SE SM V VA ZR
Phases: 

DofError: MOB4: Sublattices of (frozenset({Species('VA', 'VA1'), Species('TI', 'TI1'), Species('MO', 'MO1')}), frozenset({Species('VA', 'VA1'), Species('B', 'B1')})) contains only VA (VACUUM) constituents

In [ ]:
gibbs_energy = '???'

# Текущие результаты:

In [ ]:
print('Список компонентов:')
print(components)

In [ ]:
print('Все фазы для компонентов из списка:')
print(getAllPhases(components_phases))

In [ ]:
int_mask = 0
for phases in components_phases.values():
    int_mask |= phases
print(f'Все фазы для компонентов из списка(бинарные): {bin(int_mask)}')

In [84]:
print('Параметры расчета:')
print(f'Давление {P}, Температура {temp}, Количестов вещества по {amount} для компоненты')

Параметры расчета:
Давление 101325, Температура 1200, Количестов вещества по 0.1111111111111111 для компоненты


In [85]:
print('Значения функций для компонентов:')
print(function_value)

Значения функций для компонентов:
{'GHSERHG': -112210.200050902, 'GBCCRH': 13360.0 + GHSERRH, 'GLIQAU': 1288.9608 + GHSERAU, 'GRHOMBHG': -104149.498678239, 'GHCPAU': 2160.75 + GHSERAU, 'GLIQZR': 7251.2987626752 + GHSERZR, 'GBCCMZ': 0.0, 'GFCCZR': 6520.0 + GHSERZR, 'GHSERV': -57463.158455921, 'GHSERZR': -68894.6050887857, 'GFCCGD': 500.0 + GHSERGD, 'U': None, 'GLAVAL': 5000.0 + GHSERAL, 'GHSERAL': -54976.3276402473, 'GBCCGD': -104573.24219628, 'GBCCCZ': -5725.0, 'GHSERVZ': -57463.158455921, 'GHSERAU': -77272.6856248379, 'GFE': None, 'GBCCAU': 2930.0 + GHSERAU, 'GBCCZR': -69104.2656226965, 'GBCCAL': 4307.4 + GHSERAL, 'GLIQV': 9415.59444185221 + GHSERV, 'GHCPHG': -103109.498678239, 'GHSERGD': -105270.77857148, 'GHCPAL': 3321.0 + GHSERAL, 'LB': None, 'GLIQRH': -48315.478673478, 'GLIQSE': 0, 'GFCCHG': 10046.6 + GHSERHG, 'GB': None, 'GBCCL': None, 'GLIQAL': -57996.2850123662, 'GHSERRH': 0, 'GHSERSE': 0, 'GHCPRH': 2400.0 + GHSERRH, 'GFENI': None, 'GLIQGD': -102198.298584263}


In [86]:
print('Значение энергии Гиббса для заданых параметров:')
print(gibbs_energy)

Значение энергии Гиббса для заданых параметров:
???
